In [6]:
import torch
import random
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
from datasets import load_dataset

In [2]:
model_path = "Qwen/Qwen3-8B" 
quant_path = "./Qwen3-8B-AWQ-MMLU"
quant_config = { 
    "zero_point": True, 
    "q_group_size": 128, 
    "w_bit": 4, 
    "version": "GEMM" 
}

In [3]:
def load_mmlu_for_awq(tokenizer, n_samples=128):
    print("Loading MMLU dataset...")
    # Загружаем 'all' конфиг, сплит validation для скорости (он меньше), 
    # но можно взять и auxiliary_train, если нужен большой объем.
    # stream=True позволяет не скачивать гигабайты сразу.
    dataset = load_dataset("cais/mmlu", "all", split="validation", streaming=True)
    
    samples = []
    # Конвертируем итератор в список, чтобы можно было рандомизировать, 
    # но берем с запасом (например, 2000), чтобы выбрать разные темы.
    buffer = []
    print("Fetching examples from stream...")
    for i, example in enumerate(dataset):
        buffer.append(example)
        if i >= 2000: break
    
    # Перемешиваем, чтобы в калибровку попали разные темы (химия, история, право),
    # а не только первая по алфавиту.
    random.shuffle(buffer)
    selected_data = buffer[:n_samples]
    
    print(f"Formatting {len(selected_data)} samples with Chat Template...")
    
    options = ["A", "B", "C", "D"]
    
    for ex in selected_data:
        # Формируем тело вопроса
        question = ex['question']
        choices = ex['choices']
        answer_idx = ex['answer'] # 0, 1, 2, 3
        
        prompt_text = f"{question}\n\nChoices:\n"
        for i, choice in enumerate(choices):
            prompt_text += f"{options[i]}. {choice}\n"
        prompt_text += "Answer:"
        
        # Важнейший шаг: AWQ смотрит на активации. 
        # Если вы будете использовать модель через chat template, калибровать нужно ТОЖЕ через него.
        if tokenizer.chat_template:
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt_text}
                # Мы не добавляем ответ ассистента, так как модель должна предсказать его.
                # Но для калибровки нам нужно, чтобы модель "прогнала" через себя промпт.
            ]
            text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        else:
            # Fallback для базовых моделей без чата
            text = prompt_text

        samples.append(text)
        
    return samples


In [4]:
model = AutoAWQForCausalLM.from_pretrained(
    model_path, 
    **{"low_cpu_mem_usage": True, "use_cache": False}
)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

mmlu_samples = load_mmlu_for_awq(tokenizer, n_samples=128)

Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00, 46.40it/s]


Loading MMLU dataset...
Fetching examples from stream...
Formatting 128 samples with Chat Template...


In [5]:
model.quantize(
    tokenizer, 
    quant_config=quant_config, 
    calib_data=mmlu_samples
)

model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

AWQ:  33%|███▎      | 12/36 [09:04<18:14, 45.58s/it]

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from mmlu_benchmark import MMLUEvaluator


model = AutoModelForCausalLM.from_pretrained(
    quant_path, 
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(quant_path, trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.15s/it]


In [9]:
evaluator = MMLUEvaluator(
    model=model, tokenizer=tokenizer, device="cuda",
    split="dev", per_subject_samples=10, seed=42, model_name="awq_mmlu"
)

_ = evaluator.evaluate()

  Загружена dev выборка
  Всего вопросов в dev выборке: 285
  Количество предметов: 57
Инициализация завершена. Эксперимент: awq_mmlu_dev_20251222_203751

Эксперимент: awq_mmlu_dev_20251222_203751
Модель: awq_mmlu
Всего вопросов в dev: 285
Количество предметов: 57
Промпт стиль: zero-shot


57it [01:16,  1.34s/it]

ОБЩАЯ ТОЧНОСТЬ: 0.7368 (73.68%)
Правильных ответов: 210 из 285
Оценено предметов: 57
Пиковое потребление VRAM: 3280.76 MB
